In [1]:
import geopandas as gpd
import pandas as pd
import nltk

In [11]:
#GIST = gpd.read_parquet("https://zenodo.org/records/10127597/files/GIST_v1-0.parquet?download=1")
GIST = gpd.read_parquet("../data/large_data/GIST_v1-0.parquet")
#GIST = gpd.read_file("https://zenodo.org/records/10127597/files/GIST_v1-0.geojson?download=1", driver="GeoJSON")

In [41]:
GIST.columns

Index(['PHI_ID', 'reference', 'metadata', 'lines', 'text_raw', 'text_iphi',
       'clean_text_conservative', 'clean_text_interpretive_word', 'lemmata',
       'raw_date', 'not_before', 'not_after', 'date_type', 'region_main_id',
       'region_sub_id', 'TM_ID', 'EDH_ID', 'EDCS_ID', 'EDR_ID', 'LUPA_ID',
       'ISic_ID', 'IG_ID', 'atticinscriptions_ID', 'Attalus_ID', 'TMgeo_ID',
       'x_long', 'y_lat', 'TMgeo_name', 'polis_context_name',
       'polis_context_size', 'polis_context_fame', 'urban_context',
       'urban_context_city', 'urban_context_pop_est', 'geometry'],
      dtype='object')

In [4]:
def int_to_str(number): #fucntion authored by chatGPT...
    number_str = str(number)
    comma_position = len(str(number)) % 3
    result = number_str[:comma_position]
    while comma_position < len(number_str):
        if result:
            result += ","
        result += number_str[comma_position:comma_position + 3]
        comma_position += 3
    return result

In [5]:
gist_total = int_to_str(len(GIST))
gist_total

'217,863'

In [6]:
attrs_n = str(len(GIST.columns))
attrs_n

'35'

In [15]:
GIST.tail(10)

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,x_long,y_lat,TMgeo_name,geometry,polis_context_name,polis_context_size,polis_context_fame,urban_context,urban_context_city,urban_context_pop_est
217764,353692,IK Strat. III\n1493,Caria — Stratonikeia — Börükçü — Roman period ...,9.0,ὁ δῆμος στεφανοῖ\nκαὶ θάπτι δημοσίᾳ Δράκον-\nτ...,ο δημος στεφανοι και θαπτι δημοσια δρακοντα δρ...,ὁ δῆμος στεφανοῖ καὶ θάπτι δημοσίᾳ Δράκοντα Δρ...,ὁ δῆμος στεφανοῖ καὶ θάπτι δημοσίᾳ Δράκοντα Δρ...,"['ὁ', 'δῆμος', 'στεφανόω', 'καὶ', 'θαῦμα', 'δη...",Roman period,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217765,353693,IK Strat. III\n1494,Caria — Stratonikeia — Börükçü — 3rd c. AD? — ...,7.0,Δρόσερος\nπά(λου) γ\n\nἔκτανέ με Δρόσερον καιν...,δροσερος παλου γ εκτανε με δροσερον καινοις ορ...,Δρόσερος πά γ ἔκτανέ με Δρόσερον καινοῖς ὀρχήμ...,Δρόσερος πάλου γ ἔκτανέ με Δρόσερον καινοῖς ὀρ...,"['πρότερος', 'πῶλος', 'γ', 'κράνον', 'μή', 'Δύ...",3rd c. AD?,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217766,353694,IK Strat. III\n1495,Caria — Stratonikeia — Börükçü — 3rd c. AD? — ...,3.0,Χρῦσος\nπά(λου) η.\n,χρυσος παλου η.,Χρῦσος πά η,Χρῦσος πάλου η,"['νοῦσος', 'πῶλος', 'ὁ']",3rd c. AD?,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217767,353695,IK Strat. III\n1496,Caria — Stratonikeia — Börükçü — 3rd c. AD? — ...,3.0,Χρυσόπτερος\nπά(λου) ε.\n,χρυσοπτερος παλου ε.,Χρυσόπτερος πά ε,Χρυσόπτερος πάλου ε,"['χρυσοπτερος', 'πῶλος', 'ε']",3rd c. AD?,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217768,353696,IK Strat. III\n1497,Caria — Stratonikeia — Börükçü — 3rd c. AD? — ...,8.0,Βιτᾶλις\nπά(λου) α\n\n[ὁ θ]ρ̣ασὺς ἐν πυγμαῖς Β...,βιταλις παλου α [ο θ]ρασυς εν πυγμαις βιταλιος...,Βιτᾶλις πά α ρασὺς ἐν πυγμαῖς Βιτάλιος ἐνθάδε ...,Βιτᾶλις πάλου α ὁ θρασὺς ἐν πυγμαῖς Βιτάλιος ἐ...,"['τᾶλις', 'πῶλος', 'α', 'ὁ', 'θρασύς', 'ἐν', '...",3rd c. AD?,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217769,353697,IK Strat. III\n1498,Caria — Stratonikeia — Börükçü — 3rd c. AD? — ...,3.0,Ἀμαραῖος\nπά(λου) γ {²ϛ ph.}².\n,αμαραιος παλου γ.,Ἀμαραῖος πά γ,Ἀμαραῖος πάλου ϛ ph,"['ἡμεραῖος', 'πῶλος', 'ὁ', 'ὁ']",3rd c. AD?,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217770,353698,IK Strat. III\n1499,Caria — Stratonikeia — Börükçü — 3rd c. AD? — ...,4.0,"Εὐμήλου τόδε σῆμα κραταιοῦ μουνομάχοιο,\nὃν πο...",ευμηλου τοδε σημα κραταιου μουνομαχοιο ον πολλ...,Εὐμήλου τόδε σῆμα κραταιοῦ μουνομάχοιο ὃν πολο...,Εὐμήλου τόδε σῆμα κραταιοῦ μουνομάχοιο ὃν πολλ...,"['ὅδε', 'σῆμα', 'κραταιός', 'μονόμαχος', 'ὅς',...",3rd c. AD?,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217771,353699,IK Strat. III\n1500,Caria — Stratonikeia — Yeniköy: Hankuyu Mevkii...,9.0,ἐνθάδε κεῖ-\nται ἱερεὺς\nΔήμητρος·\nὃ̣ς ἂν ἀδι...,ενθαδε κειται ιερευς δημητρος ος αν αδικησει ε...,ἐνθάδε κεῖται ἱερεὺς Δήμητρος ὃς ἂν ἀδικήσει ἐ...,ἐνθάδε κεῖται ἱερεὺς Δήμητρος ὃς ἂν ἀδικήσει ἐ...,"['ἐνθάδε', 'κεῖμαι', 'ἱερεύς', 'ὅς', 'ἄν', 'ἀδ...",ca. 425-400 BC,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217772,353700,IK Strat. III\n1501,Caria — Stratonikeia — Çatlı — ca. 50-25 BC — ...,19.0,ἐπὶ στ̣εφ̣ανηφόρου Φανίου το̣ῦ\nἈρτεμιδώ̣ρου τ...,επι στεφανηφορου φανιου του αρτεμιδωρου του με...,ἐπὶ στεφανηφόρου Φανίου τοῦ Ἀρτεμιδώρου τοῦ Με...,ἐπὶ στεφανηφόρου Φανίου τοῦ Ἀρτεμιδώρου τοῦ Με...,"['ἐπί', 'στεφανηφόρος', 'Φανουήλ', 'ποῦ', 'Ἀρτ...",ca. 50-25 BC,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN
217773,353701,IK Strat. III\n1502,Caria — Stratonikeia — Çatlı — ca. 50-25 BC — ...,31.0,[— — — — —]Λ[— — — — —]\n[κα]θ’ ὑοθεσίαν δὲ Δι...,[-----]λ[-----] [κα]θ υοθεσιαν δε διογ[ενους] ...,Λ θ’ ὑοθεσίαν δὲ Διογ ἱερεὺς Ἐπαίνετος Ἱεροκου...,Λ καθ’ ὑοθεσίαν δὲ Διογνήτου ἱερεὺς Ἐπαίνετος ...,"['ὁ', 'ὑοθεσία', 'δὲ', 'διόγνητος', 'ἱερεύς', ...",ca. 50-25 BC,...,NaN,NaN,None,None,None,NaN,NaN,None,None,NaN


In [17]:
~GIST["geometry"].is_empty

0         True
1         True
2         True
3         True
4         True
          ... 
217769    True
217770    True
217771    True
217772    True
217773    True
Length: 217863, dtype: bool

In [25]:
valid_geo_n = int_to_str((GIST["geometry"].is_valid).sum())
valid_geo_n

'180,061'

In [26]:
dated_n = int_to_str(((GIST["not_before"].notnull()) | (GIST["not_after"].notnull())).sum())
dated_n

'131,677'

In [35]:
geotemporal_n = int_to_str(((GIST["geometry"].is_valid) & ((GIST["not_before"].notnull()) | (GIST["not_after"].notnull()))).sum())
geotemporal_n

'106,898'

In [38]:
description = "The GIST dataset consists of {0} inscriptions, enriched by {1} attributes.\n\n {2} inscriptions have valid geospatial coordinates (the `geometry` attribute). This information is also used to determine the Roman urban context of each inscription (i.e. whether it is in the neighbourhood (i.e. within a 5000m buffer) of a large city, medium city, or small city or rural (>5000m to any type of city; see the attributes `urban_context`, `urban_context_city`, and `urban_context_pop`) and for their mapping on an ancient Greek polis (if there is any within the 5000m buffer; see the attributes `polis_context_name`, `polis_context_size`, and `polis_context_fame`).\n\n{3} inscriptions have a numerical date of origin expressed by means of an interval or singular year using the attributes `not_before` and `not_after`. \n\n{4} inscriptions have both geospatial coordinates and numerically expressed date of origin (see the 'is_geotemporal' attribute).)".format(
    gist_total,
    attrs_n,
    valid_geo_n,
    dated_n,
    geotemporal_n,
)
print(description)

The GIST dataset consists of 217,863 inscriptions, enriched by 35 attributes.

 180,061 inscriptions have valid geospatial coordinates (the `geometry` attribute). This information is also used to determine the Roman urban context of each inscription (i.e. whether it is in the neighbourhood (i.e. within a 5000m buffer) of a large city, medium city, or small city or rural (>5000m to any type of city; see the attributes `urban_context`, `urban_context_city`, and `urban_context_pop`) and for their mapping on an ancient Greek polis (if there is any within the 5000m buffer; see the attributes `polis_context_name`, `polis_context_size`, and `polis_context_fame`).

131,677 inscriptions have a numerical date of origin expressed by means of an interval or singular year using the attributes `not_before` and `not_after`. 

106,898 inscriptions have both geospatial coordinates and numerically expressed date of origin (see the 'is_geotemporal' attribute).)


In [9]:
GIST[GIST["PHI_ID"].isin([333386, 230234, 2708])]

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID,TMgeo_ID,x_long,y_lat,TMgeo_name,geometry
2313,2708,IG II²\n490,Att. — stoich. 33 — 303/2,9.0,[ἐπὶ Λεωστράτου ἄρχοντος] ἐπὶ τῆς Κεκροπ-\n[ίδ...,[επι λεωστρατου αρχοντος] επι της κεκροπ[ιδος ...,ἐπὶ τῆς Κεκροπι Διόφαντος Διραμμάτευεν Ἀνέκα ἐ...,ἐπὶ Λεωστράτου ἄρχοντος ἐπὶ τῆς Κεκροπίδος ὀγδ...,"{'data': ['ἐπί', 'εὔστρωτος', 'ἄρχων', 'ἐπί', ...",303/2,...,{'data': []},{'data': ['ISic002719']},{'data': []},{'data': []},{'data': []},{'data': ['2210']},15.282327,37.071289,Syrakousai,POINT (15.28233 37.07129)
97001,230234,SEG\n46:129,Attica — Athens — 302/3 BC — IG II² 490 — Horo...,1.0,Διόφαντος Δι[ονυσοδώρου Φηγούσιος] {²⁷Διόφαντο...,διοφαντος δι[ονυσοδωρου φηγουσιος].,Διόφαντος Δι,Διόφαντος Διονυσοδώρου Διόφαντος Διοπείθου Μυρ...,"{'data': ['διύφαντος', 'διύφαντος', 'Συρακούσι...",302/3 BC,...,{'data': []},{'data': ['ISic002719']},{'data': []},{'data': []},{'data': []},{'data': ['2210']},15.282327,37.071289,Syrakousai,POINT (15.28233 37.07129)
198856,333386,SEG\n46:1290,Sikelia — Syrakousai (Siracusa) — Cat. S. Giov...,5.0,[— — — — —]\n[—]․[— — —]\nΗΜ[— — — —]\nv ΕΝ[— ...,[-----] [-]-[---] 0[----] εν[---] [-----].,ΗΜ ΕΝ,ΗΜ ΕΝ,"{'data': ['ΗΜ', 'ΕΝ']}",Christian period,...,{'data': []},{'data': ['ISic002719']},{'data': []},{'data': []},{'data': []},{'data': ['2210']},15.282327,37.071289,Syrakousai,POINT (15.28233 37.07129)


In [7]:
GIST[GIST["PHI_ID"]==230234]

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID,TMgeo_ID,x_long,y_lat,TMgeo_name,geometry
97001,230234,SEG\n46:129,Attica — Athens — 302/3 BC — IG II² 490 — Horo...,1.0,Διόφαντος Δι[ονυσοδώρου Φηγούσιος] {²⁷Διόφαντο...,διοφαντος δι[ονυσοδωρου φηγουσιος].,Διόφαντος Δι,Διόφαντος Διονυσοδώρου Διόφαντος Διοπείθου Μυρ...,"{'data': ['διύφαντος', 'διύφαντος', 'Συρακούσι...",302/3 BC,...,{'data': []},{'data': ['ISic002719']},{'data': []},{'data': []},{'data': []},{'data': ['2210']},15.282327,37.071289,Syrakousai,POINT (15.28233 37.07129)


In [5]:
GIST[GIST["ISic_ID"].apply(lambda x:  len(x["data"]) > 0)]

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID,TMgeo_ID,x_long,y_lat,TMgeo_name,geometry
2313,2708,IG II²\n490,Att. — stoich. 33 — 303/2,9.0,[ἐπὶ Λεωστράτου ἄρχοντος] ἐπὶ τῆς Κεκροπ-\n[ίδ...,[επι λεωστρατου αρχοντος] επι της κεκροπ[ιδος ...,ἐπὶ τῆς Κεκροπι Διόφαντος Διραμμάτευεν Ἀνέκα ἐ...,ἐπὶ Λεωστράτου ἄρχοντος ἐπὶ τῆς Κεκροπίδος ὀγδ...,"{'data': ['ἐπί', 'εὔστρωτος', 'ἄρχων', 'ἐπί', ...",303/2,...,{'data': []},{'data': ['ISic002719']},{'data': []},{'data': []},{'data': []},{'data': ['2210']},15.282327,37.071289,Syrakousai,POINT (15.28233 37.07129)
3051,3446,IG II²\n1231,Att. — stoich. 25 — fin. s. IV a.,16.0,θ[εοί].\nΠρόφαν[τος εἶπεν· ἐπειδὴ Τληπό]-\nλεμ...,θ[εοι]. προφαν[τος ειπεν επειδη τληπο]λεμος α[...,θ Πρόφανλεμος Αλῶς καὶ φις τῶν ἱερῶν ἐὸ τοῦ Πλ...,θεοί Πρόφαντος εἶπεν ἐπειδὴ Τληπόλεμος Α καλῶς...,"{'data': ['θεός', 'γράφω', 'λέγω', 'ἐπειδή', '...",fin. s. IV a.,...,{'data': []},{'data': ['ISic003055']},{'data': []},{'data': ['93']},{'data': []},{'data': ['22311']},14.954018,38.467383,Meligounis,POINT (14.95402 38.46738)
4713,5109,IG II²\n2855,Att. — non-stoich. — c. 258/7,7.0,Δεινίας Κηφ[ισο]δότου Βουτάδης\nθεσμοθετήσας ἐ...,δεινιας κηφ[ισο]δοτου βουταδης θεσμοθετησας επ...,Δεινίας Κηφδότου Βουτάδης θεσμοθετήσας ἐπὶ Φει...,Δεινίας Κηφισοδότου Βουτάδης θεσμοθετήσας ἐπὶ ...,"{'data': ['νεανίας', 'Κηφισός', 'Βουτάδης', 'θ...",c. 258/7,...,{'data': []},{'data': ['ISic003390']},"{'data': ['IG II_III³ 4', ' 99']}",{'data': ['2855']},{'data': []},{'data': ['364']},23.723985,37.972747,Athenai,POINT (23.72399 37.97275)
34593,140290,IG XIV\n1,Sikelia — Syrakousai (Siracusa) — early 6th c....,1.0,Κλεομ[․․]εˉς ∶ ἐποίεˉσε τὀˉπέλοˉνι {²⁶το͂ι Ἀπέ...,κλεομ[--]ες. εποιεσε τοπελονι. ο κνιδιειδα. κε...,Κλεομεˉς ἐποίεˉσε τὀˉπέλοˉνι ℎο Κνιδιείδα κἐπι...,Κλεομεˉς ἐποίεˉσε το͂ι Ἀπέλοˉνι ℎο Κνιδιείδα κ...,"{'data': ['ὀλεμεύς', 'ἐκεῖσε', 'ὁ', 'ἀπολογία'...",early 6th c. BC,...,{'data': []},{'data': ['ISic000822']},{'data': []},{'data': []},{'data': []},{'data': ['2210']},15.282327,37.071289,Syrakousai,POINT (15.28233 37.07129)
34594,140291,IG XIV\n2,Sikelia — Syrakousai (Siracusa) — 269-215 BC,3.0,βασιλέος ἁγε[μόνος]\nἹέρωνος Ἱεροκλέος.\nΣυρακ...,βασιλεος αγε[μονος] ιερωνος ιεροκλεος. συρακοσ...,βασιλέος ἁγε Ἱέρωνος Ἱεροκλέος Συρακόσιοι θεοῖ...,βασιλέος ἁγεμόνος Ἱέρωνος Ἱεροκλέος Συρακόσιοι...,"{'data': ['βασιλεύς', 'ἡγεμονίς', 'ἱεροκόμος',...",269-215 BC,...,{'data': []},{'data': ['ISic000823']},{'data': []},{'data': []},{'data': ['s427']},{'data': ['2210']},15.282327,37.071289,Syrakousai,POINT (15.28233 37.07129)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204234,340069,SEG\n47:1458[1],Sikelia — Selinous — ca. 500-450 BC — IGLPaler...,1.0,οἴμ[ο]ι ὀ͂ [—]νδα {²⁷[Ἔ]νδα(?)}²⁷ οἴμοι.,οιμ[ο]ι ο [-]νδα οιμοι.,οἴμι ὀ͂ νδα οἴμοι,οἴμοι ὀ͂ Ἔνδα οἴμοι,{'data': ['ἄνδα']},ca. 500-450 BC,...,{'data': []},{'data': ['ISic001572']},{'data': []},{'data': []},{'data': []},{'data': ['22286']},12.825120,37.582841,Selinous,POINT (12.82512 37.58284)
204235,340070,SEG\n47:1458[2],Sikelia — Selinous — ca. 500-450 BC — IGLPaler...,1.0,οἴμ[ο]ι ὀ͂ [Νε]νδα(?) οἴμοι.,οιμ[ο]ι ο [νε]νδα οιμοι.,οἴμι ὀ͂ νδα οἴμοι,οἴμοι ὀ͂ Νενδα οἴμοι,"{'data': ['οἴμοι', 'ὀ͂', 'Νενδα', 'οἴμοι']}",ca. 500-450 BC,...,{'data': []},{'data': ['ISic001572']},{'data': []},{'data': []},{'data': []},{'data': ['22286']},12.825120,37.582841,Selinous,POINT (12.82512 37.58284)
204242,340077,SEG\n47:1464,Sikelia — Tauromenion (Taormina) — late 3rd/2n...,15.0,"SEG 26.1123, frg. II, l. 1; new frgs. IV and V...",[φαιδ]ων ηλειος [--]ο [--]λι αναξιμανδρος πραξ...,SEG frg II l new frgs IV and V ων Ἠλεῖος Ο ΛΙ ...,SEG frg II l new frgs IV and V H Blanck Φαίδων...,"{'data': ['ὁ', 'ὁ', 'ὁ', 'ὁ', 'ὁ', 'ὁ', 'ὁ', '...",late 3rd/2nd c. BC,...,{'data': []},{'data': ['ISic000613']},{'data': []},{'data': []},{'data': ['inscr_166.html#C']},{'

In [10]:
ISic = pd.read_csv("../data/ISic.csv")
ISic

,Id,Date,After,Before,Ancient,Modern,Material,Object Type,Inscription Type,Execution Type,Language,Museum,Status,Trismegistos,EDR,PHI,EDCS
0,ISic000001,between later 1st and 3rd century CE,0050,0300,NaN,Caltanissetta,marble,plaque,funerary,chiselled,Latin,Museo Archeologico Regionale Antonino Salinas,draft,491696,NaN,NaN,21900531
1,ISic000002,3rd century CE,0201,0300,Catina,Catania,marble,plaque,funerary,chiselled,Latin,Museo Archeologico Regionale Antonino Salinas,draft,491568,141180,NaN,21900397
2,ISic000003,Bivona 2nd — 3rd century CE Manganaro on the b...,0101 0001 0101,0300 0100 0200,Catina,Catania,marble,plaque,funerary,chiselled,Latin,Museo Archeologico Regionale Antonino Salinas,draft,491590,141191,NaN,21900419
3,ISic000004,100—150 CE,0100,0150,Centuripae,Centuripe,marble,plaque,building,chiselled,Latin,Museo Archeologico Regionale Antonino Salinas,draft,491500,NaN,NaN,21900323
4,ISic000005,Imperial,0001,0300,Centuripae,Centuripe,marble,NaN,funerary,chiselled,Latin,Museo Archeologico Regionale Antonino Salinas,draft,491502,NaN,NaN,21900325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4575,ISic030314,480—450 BCE,-0480,-0450,Kamarina,Santa Croce Camerina,lead,tessera,tessera (civic),engraved,Ancient Greek,Museo Archeologico Regionale di Kamarina,draft,NaN,NaN,NaN,NaN
4576,ISic030315,480—450 BCE,-0480,-0450,Kamarina,Santa Croce Camerina,lead,tessera,tessera (civic),engraved,Ancient Greek,Museo Archeologico Regionale di Kamarina,draft,NaN,NaN,NaN,NaN
4577,ISic030317,first half of the 6th century BCE,-0599,-0550,NaN,Monte Casasia,lead,lamina,magical,engraved,Ancient Greek,Museo Archeologico Regionale di Kamarina,draft,NaN,NaN,NaN,NaN
4578,ISic030319,second half of the 4th century BCE,-0350,-0300,Palikè,Area archeologica di Palikè,bronze,belt,votive,punched,Ancient Greek,Museo Archeologico Regionale Paolo Orsi,draft,NaN,NaN,NaN,NaN


In [22]:
set(ISic[ISic["Language"].str.contains("Greek", na=False) & ~(ISic["Language"].str.contains("Latin", na=False))]["Language"])

{'Ancient Greek',
 'Ancient Greek\n                            Sikel',
 'Ancient Greek and Punic',
 'Ancient GreekElymian',
 'Ancient GreekSikel',
 'ElymianAncient Greek',
 'GreekSikel',
 'Possibly Ancient Greek',
 'PunicAncient Greek',
 'SikelAncient Greek',
 'SikelGreek'}

In [15]:
(ISic["Language"]=="Greek").sum()

0

In [25]:
nltk.FreqDist(ISic["Inscription Type"].tolist()).most_common()

[('funerary', 1986),
 ('unknown', 731),
 (nan, 445),
 ('dedication', 252),
 ('honorific', 232),
 ('tessera (civic)', 166),
 ('ownership', 92),
 ('building', 84),
 ('ownership (?)', 77),
 ('votive', 74),
 ('defixio', 69),
 ('production or ownership (?)', 67),
 ('decree', 23),
 ('seat\n                        inscription\n                    ', 21),
 ('accounts', 18),
 ('terminus', 16),
 ('prayer', 13),
 ('honorific; building', 11),
 ('list', 10),
 ('public document', 10),
 ('votive (?)', 10),
 ('funerary; honorific', 9),
 ('building; honorific', 7),
 ('greeting', 7),
 ('graffiti', 7),
 ('dedication; list of magistrates', 6),
 ('private (graffito), label indicating the content of the vessel (?)', 6),
 ('honorific; funerary', 5),
 ('votive inscription', 4),
 ('owner inscription', 4),
 ('milestone', 3),
 ('list of\n                        magistrates\n                    ', 3),
 ('cadastral', 3),
 ('epigram', 3),
 ('dedication; honorific', 3),
 ('abecedarium', 3),
 ('honorific; dedication'

In [26]:
GIST[GIST["atticinscriptions_ID"].apply(lambda x:  len(x["data"]) > 0)]

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID,TMgeo_ID,x_long,y_lat,TMgeo_name,geometry
0,1,IG I³\n1,Att. — Ath.: Akr. — stoich. 35 — c. 510-500 a....,12.0,ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ...,εδοχσεν τοι δεμοι τ[ος ε σ]αλαμ[ινι κλεροχ]ος ...,ἔδοχσεν το͂ι δέμοι ταλαμος οἰκε͂ν ἐᾶ Σαλαμῖνι ...,ἔδοχσεν το͂ι δέμοι τὸς ἐ Σαλαμῖνι κλερόχος οἰκ...,"{'data': ['ἔδοχσεν', 'το͂ι', 'δέμοι', 'τὸς', '...",c. 510-500 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 1']},{'data': ['1672']},{'data': []},{'data': ['364']},23.723985,37.972747,Athenai,POINT (23.72399 37.97275)
1,2,IG I³\n2,Att. — non-stoich. — c. 500 a.,14.0,[․․8-9․․․]ν̣ βολ — — — — — — — — — —\n[․6-7․․]...,[--------]ν βολ ---------- [------] α εκον ---...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,ν βολ α ℎεκον σιον γνοσθε͂ι δὲ ν ἀτεχνος μὲ π ...,"{'data': ['ν', 'βοῦς', 'α', 'θύος', 'γνώστης',...",c. 500 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 2']},{'data': ['1707']},{'data': []},{'data': ['373']},NaN,NaN,None,None
2,3,IG I³\n3,Att. — stoich. 21 — 490-480 a.,13.0,[․]αρ[․․․․]ι ℎερακλειο[․․5․․]\n[․]αρ̣ο#⁷[․] τι...,[-]αρ[----]ι ερακλειο[-----] [-]αρο [-] τιθενα...,αρι ℎερακλειο αρο τιθέναι τὸς ἀέτας τριάκοντα ...,αρι ℎερακλειο αρο τιθέναι τὸς ἀθλοθέτας τριάκο...,"{'data': ['Ἤρ', 'εἰσπλέω', 'Ἀθῆναι', 'τὶς', 'ἀ...",490-480 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 3']},{'data': ['ii-1b']},{'data': []},{'data': ['373']},NaN,NaN,None,None
3,4,IG I³\n4,Att. — stoich. 38 — 485/4 a.,56.0,[․․․․․․․․․․․․․․․․․․38․․․․․․․․․․․․․․․․․․]\n[․․․...,[--------------------------------------] [----...,δέ τις αν ἒ φρορὰν μ ντέκοντα δχμὰς τ ας ℎες π...,ἐὰν δέ τις αν ἒ φρορὰν μὲ πεντέκοντα δραχμὰς τ...,"{'data': ['ἐάν', 'τις', 'ἐν', 'ἒ', 'φρουρά', '...",485/4 a.,...,{'data': []},{'data': []},"{'data': ['IG I³ 4(A)', 'IG I³ 4(B)']}",{'data': ['1692']},{'data': []},{'data': ['364']},23.723985,37.972747,Athenai,POINT (23.72399 37.97275)
4,5,IG I³\n5,Att. — c. 500 a.,6.0,[ἔδοχσε]ν [⋮ τε͂ι βολε͂ι] ⋮ καὶ [τ]ο͂ι δέμοι ⋮...,[εδοχσε]ν [ τει βολει] και [τ]οι δεμοι οτε παρ...,ν καὶ ο͂ι δέμοι ℎότε Παραιβάτες λεια θν τὸς ℎι...,ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι ℎότε Παραιβ...,"{'data': ['δοκέω', 'βουλή', 'καὶ', 'ὁ', 'δῆμος...",c. 500 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 5']},{'data': ['1284']},{'data': []},{'data': ['10345']},23.541498,38.044135,Eleusis,POINT (23.54150 38.04414)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212188,348047,"IG II³,1\n1134",Attica — Athens — Akropolis? — undated — IG II...,7.0,[— — — — — — — — — — — — — — —]\n[— — c.11 — —...,[---------------] [-----------]λνιης[------] κ...,ΛΝΙΗΣ καὶ τοὺς ἐκγόνους αὐτοῦ ΙΣ ἀναγράψαι τόδ...,c ΛΝΙΗΣ καὶ τοὺς ἐκγόνους αὐτοῦ ΙΣ ἀναγράψαι τ...,"{'data': ['ὁ', 'καὶ', 'τὶς', 'ἔκγονος', 'αὐτός...",IG II² 873,...,{'data': []},{'data': []},"{'data': ['IG II_III³ 1', ' 1134']}",{'data': ['1134']},{'data': []},{'data': ['373']},NaN,NaN,None,None
213839,349731,"IG XII,4\n1:161",Kos Isl. — Kos — Asklepieion — early 2nd c. BC...,12.0,"vacat 0,03\n[Πο]λύδικος Στρατοκλ[έους — — — —]...",[πο]λυδικος στρατοκλ[εους ------] ειπεν επειδη...,λύδικος Στρατοκλ εἶπεν ἐπειδὴ Κλεομ νουν καὶ χ...,Πολύδικος Στρατοκλέους εἶπεν ἐπειδὴ Κλεομ Κῶιο...,"{'data': ['πολύδικος', 'στρατός', 'λέγω', 'ἐπε...",early 2nd c. BC,...,{'data': []},{'data': []},"{'data': ['IG XII 4', ' 1', ' 161', 'IG II_III...",{'data': ['1240']},{'data': []},{'data': ['2796']},27.287424,36.893269,Kos,POINT (27.28742 36.89327)
215917,351823,"Milet VI,3\n1032",Ionia — Miletos — ca. 200 BC — Chiron 28 (1998...,9.0,[— c.4 —]Ι̣ τῶι δή̣μωι τῶ̣[ι Μιλησίων — — — — ...,[----]ι τωι δημωι τω[ι μιλησιων --------] ---ε...,Ι τῶι δήμωι τῶ ΕΙΝ Ἀρίσταιχμον Τ ΝΟΣ γνώμην δὲ...,c Ι τῶι δήμωι τῶι Μιλησίων ΕΙΝ Ἀρίσταιχμον Τ Ν...,"{'data': ['ὁ', 'ὁ', 'τι', 'δῆμος', 'τι', 'Μνάσ...",ca. 200 BC,...,{'data': []},{'data':

In [ ]:
ἔδοχσεν το͂ι δέμοι· τ̣[ὸς ἐ Σ]αλαμ̣[ῖνι κλερόχ]ος\nοἰκε͂ν ἐᾶ Σαλαμῖνι [․․5․․]λεν [․․․7․․․ Ἀθέ]νε-\nσι τελε͂ν καὶ στρατ[εύεσθ]αι ⋮ τ̣[ὰ δ’ ἐ Σαλαμῖνι] μ-\nὲ μι[σθ]ο͂ν, ἐὰ μὲ οἰκ[․․․7․․․]ο[․ μισθόμε...

In [29]:
GIST["lemmata"]

0         {'data': ['ἔδοχσεν', 'το͂ι', 'δέμοι', 'τὸς', '...
1         {'data': ['ν', 'βοῦς', 'α', 'θύος', 'γνώστης',...
2         {'data': ['Ἤρ', 'εἰσπλέω', 'Ἀθῆναι', 'τὶς', 'ἀ...
3         {'data': ['ἐάν', 'τις', 'ἐν', 'ἒ', 'φρουρά', '...
4         {'data': ['δοκέω', 'βουλή', 'καὶ', 'ὁ', 'δῆμος...
                                ...                        
217858    {'data': ['ἀριστερός', 'ἱεροκόμος', 'ποῦ', 'Θε...
217859                   {'data': ['Ἄρειος', 'Ἀρτεμίσιον']}
217860                           {'data': ['καὶ', 'κριός']}
217861    {'data': ['ὡς', 'μανία', 'Πέργαμος', 'πού', 'χ...
217862    {'data': ['ὁ', 'στρατιώτης', 'βράχoς', 'λωτοφά...
Name: lemmata, Length: 217863, dtype: object

In [31]:
GIST["lemmata"] = GIST["lemmata"].apply(lambda x: x["data"])

In [32]:
def find_word(word, lemmata):
    try:
        return word in lemmata
    except:
        return False
word = 'δραχμός'
GIST["lemmata"].apply(lambda x: find_word(word, x))

0         False
1         False
2         False
3         False
4         False
          ...  
217858    False
217859    False
217860    False
217861    False
217862    False
Name: lemmata, Length: 217863, dtype: bool

In [34]:
GIST[GIST["lemmata"].apply(lambda x: find_word(word, x))]

,PHI_ID,reference,metadata,lines,text_raw,text_iphi,clean_text_conservative,clean_text_interpretive_word,lemmata,raw_date,...,LUPA_ID,ISic_ID,IG_ID,atticinscriptions_ID,Attalus_ID,TMgeo_ID,x_long,y_lat,TMgeo_name,geometry
5,6,IG I³\n6,Att. — stoich. 23/11 — ante 460 a.,160.0,— — — — — — — — — — — — —\n[․․․․․․15․․․․․․․] δ...,------------- [---------------] δραχμεισ[ι ---...,δραχμε͂ιστες τὸς ιο μενος δεμο το͂ν πόλεο οκε͂...,δραχμε͂ισι τες τὸς ιο μενος δεμο το͂ν πόλεον δ...,"[δραχμός, τε, τὶς, μόνος, δῆμος, ὁ, παλέω, δοκ...",ante 460 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 6']},{'data': ['6']},{'data': []},{'data': ['10345']},23.541498,38.044135,Eleusis,POINT (23.54150 38.04414)
18,19,IG I³\n19,Att. — stoich. 28 — c. 450/49 a.,13.0,[․․․․ ἐ͂ναι] δ α[ὐτὸν πρόχσενον Ἀθενα]-\n[ίον...,[---- ε ναι] δ α[υτον προχσενον αθεναιον και ε...,δ αὐεργέτε αι Ἀχελοΐοατὰ τούτον Ἀέμαρχον πρυτὲ...,ἐ͂ναι δ αὐτὸν πρόχσενον Ἀθεναίον καὶ εὐεργέτε...,"[εἰμί, δ, αὐτός, πράσσω, φθινάς, καὶ, εὐεργέτη...",c. 450/49 a.,...,{'data': []},{'data': []},{'data': []},{'data': []},{'data': []},{'data': ['373']},NaN,NaN,None,None
40,41,IG I³\n41,Att. — stoich. 39? — c. 446/5 a.,124.0,v. unus perditus est\n[․․․․․․․․․․․․․․․33․․․․․․...,[---------------------------------]τε[----] [-...,unus perditus est τε ονι σθα ΟΕΝΑ δὲ βοε ι χιλ...,unus perditus est τε ονι σθα ΟΕΝΑ δὲ βοεθ ι χι...,"[ὁ, ὁ, τις, τε, σιά, δὲ, βοάω, ὁ, φίλος, Ναιμά...",c. 446/5 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 41']},{'data': ['3']},{'data': []},{'data': ['373']},NaN,NaN,None,None
59,60,IG I³\n59,Att. — stoich. 69-74 — c. 430 a.,69.0,[ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι· — — — ὶς ...,[εδοχσεν τει βολει και τοι δεμοι --- ις επρυτα...,αντοκλε͂ ἐγρα Ἀθεναίας μετὰ το θεναίας καὶ τὲν...,ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι ὶς ἐπρυτάνε...,"[δοκέω, βουλή, καὶ, ὁ, δῆμος, πρυτανεία, Ἰατρο...",c. 430 a.,...,{'data': []},{'data': []},{'data': []},{'data': []},{'data': []},{'data': ['373']},NaN,NaN,None,None
83,84,IG I³\n82,Att. — stoich. 61 — 421/0 a.,45.0,[θ]ε[οί]·\n[Προκλε͂ς] Ἀτάρβο Ε[ὐονυμεὺς]\n[ἐγρ...,[θ]ε[οι] [προκλες] αταρβο ε[υονυμευς] [εγραμμα...,ε Ἀτάρβο Ε ευε ἐπὶ Ἀριστ έμοι ℎιπποθοντὶτίον ε...,θεοί Προκλε͂ς Ἀτάρβο Εὐονυμεὺς ἐγραμμάτευε ἐπὶ...,"[θεός, προπλέω, τάρβος, εὐθυμέω, γραμματεύς, ἐ...",421/0 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 82']},{'data': ['1304']},{'data': []},{'data': ['373']},NaN,NaN,None,None
103,104,IG I³\n102,Att. — stoich. 36 — 410/09 a.,48.0,[ἐπὶ Γλαυκί]ππο ἄ[ρ]χον[τ]ος.\n[Λόβον ἐκ] Κεδο...,[επι γλαυκι]ππο α[ρ]χον[τ]ος. [λοβον εκ] κεδον...,ππο ἄχονος Κεδο͂ν ἐγραμμάτευε βολε͂ι καὶ το͂ι ...,ἐπὶ Γλαυκίππο ἄρχοντος Λόβον ἐκ Κεδο͂ν ἐγραμμά...,"[ἐπί, ἄρχων, ἆσσον, ἐκ, Κεῖος, γραμματεύς, δοκ...",410/09 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 102']},{'data': ['182']},{'data': []},{'data': ['373']},NaN,NaN,None,None
106,107,IG I³\n105,Att. — stoich. 66 — c. 409 a.,61.0,[․․․․․․․․․20․․․․․․․․․]#⁷ — — — — — — — — — — —...,[--------------------] -----------------------...,Ο ΙΟ ΣΕΕΠΟ ΙΛ ΜΙλος ττὸν α ΑΣ ΟΙΟ ΙΟ λνο πολεμ...,Ο ΙΟ ΣΕΕΠΟ ΙΛ ΜΙ οὐδὲ δέσο Ἀθεναίον οὐδὲ ℎένα ...,"[ὁ, οὐδέ, δέω, φθινάς, οὐδέ, εἷς, ὅς, ἄν, ἐγγυ...",c. 409 a.,...,{'data': []},{'data': []},{'data': ['IG I³ 105']},{'data': ['105']},{'data': []},{'data': ['373']},NaN,NaN,None,None
132,133,IG I³\n130,Att. — stoich. 60? — c. 432 a.,24.0,θε[οί]·\nἔδοχσεν τε͂ι βολε͂ι καὶ τ[ο͂ι] δέμοι·...,θε[οι] εδοχσεν τει βολει και τ[οι] δεμοι αντ[ι...,θε ἔδοχσεν τε͂ι βολε͂ι καὶ τ δέμοι Ἀντευε Στρά...,θεοί ἔδοχσεν τε͂ι βολε͂ι καὶ το͂ι δέμοι Ἀντιοχ...,"[θεός, δοκέω, βουλή, καὶ, ὁ, δῆμος, Ἀντίοχος, ...",c. 432 a.,...,{'data': []},{'data': []},{'data': []},{'data': []},{'data': []},{'data': ['373']},NaN,NaN,None,None
250,252,IG I³\n243,Att. — stoich. — 480-450,151.0,[ἔδοχσεν τε͂ι — — — — — — β]ολε͂ι ⋮ ἐπ̣-\n[ὶ —...,[εδοχσεν τει ------ β]ολει επ[ι --- αρχοντος -...,ολε͂ι ἐπ ἄλλει ε ἐπορ ιε κλ εο λ ακιαν καὶ μάν...,ἔδοχσεν τε͂ι βολε͂ι ἐπὶ ἄρχοντος ἄλλει ε ἐπορ ...,"[δοκέω, βουλή, ἐπί, ἄρχων, ἄλλος, ε, πού, εἰ